### Your name:

<pre>Rishab Kamshetty</pre>

### Collaborators:

<pre>Ashutosh Deowanshi</pre>


In [0]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

def rmse(a, b):
    return np.sqrt(np.mean((a-b)**2))
pd.options.display.max_columns = None
# to make this notebook's output identical at every run
np.random.seed(0)

Markdown referece can be found here:
    http://nestacms.com/docs/creating-content/markdown-cheat-sheet

**Q1. Why would it be a problem if our training set and test set are the same.**

<pre>If the training set and test set are same then the model will give always give good result for the same set of data. And the would not generalize for the new set of test data. </pre>


Open the housing data


In [149]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [150]:
sample_incomplete_rows = housing[housing.isnull().any(axis=1)].head()
sample_incomplete_rows

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
290,-122.16,37.77,47.0,1256.0,NaN,570.0,218.0,4.3750,161900.0,NEAR BAY
341,-122.17,37.75,38.0,992.0,NaN,732.0,259.0,1.6196,85100.0,NEAR BAY
538,-122.28,37.78,29.0,5154.0,NaN,3741.0,1273.0,2.5762,173400.0,NEAR BAY
563,-122.24,37.75,45.0,891.0,NaN,384.0,146.0,4.9489,247100.0,NEAR BAY
696,-122.10,37.69,41.0,746.0,NaN,387.0,161.0,3.9063,178400.0,NEAR BAY


In [0]:
# Use the sklearn imputer class, select median as method 
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

In [152]:
[(x, y) for x, y in zip(housing.isna().sum(), housing.isna().sum().index) if x > 0]

[(207, 'total_bedrooms')]

In [0]:
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median, inplace=True)

In [154]:
# remove text data, cannot impute 
# create copy 
housing_num = housing.drop('ocean_proximity', axis=1)

# run imputer on numerical data 
imputer.fit(housing_num)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [155]:
# peak into estimators
imputer.statistics_

array([-1.1849e+02,  3.4260e+01,  2.9000e+01,  2.1270e+03,  4.3500e+02,
        1.1660e+03,  4.0900e+02,  3.5348e+00,  1.7970e+05])

In [156]:
# sanity check values 
housing_num.median().values

array([-1.1849e+02,  3.4260e+01,  2.9000e+01,  2.1270e+03,  4.3500e+02,
        1.1660e+03,  4.0900e+02,  3.5348e+00,  1.7970e+05])

In [0]:
X = imputer.transform(housing_num)

In [158]:
# convert back into dataframe 
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                          index = housing.index.values)
housing_tr.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [159]:
# look at imputed data 
housing_tr.loc[sample_incomplete_rows.index.values]

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
290,-122.16,37.77,47.0,1256.0,435.0,570.0,218.0,4.3750,161900.0
341,-122.17,37.75,38.0,992.0,435.0,732.0,259.0,1.6196,85100.0
538,-122.28,37.78,29.0,5154.0,435.0,3741.0,1273.0,2.5762,173400.0
563,-122.24,37.75,45.0,891.0,435.0,384.0,146.0,4.9489,247100.0
696,-122.10,37.69,41.0,746.0,435.0,387.0,161.0,3.9063,178400.0


In [160]:
# clean index this time
housing_tr = pd.DataFrame(X, columns=housing_num.columns)
housing_tr.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [161]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20640 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_value    20640 non-null float64
ocean_proximity       20640 non-null object
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [162]:
housing.isnull().values.any()

False

### Build full pipeline for the data analysis following the example of the notebook we went through in class.
 Hint: the main part requested to change is the algorithm used (Lasso regression)

If you want to learn more about the Lasso regression, see resources below:
- http://scikit-learn.org/stable/modules/linear_model.html#lasso
- https://www.analyticsvidhya.com/blog/2016/01/complete-tutorial-ridge-lasso-regression-python/

#### Considerations for building pipeline:

- Split data into training and testing sets below.
- Convert all categorical data to one-hot vectors below
- Normalize all non-categorical data 
-  Perform Lasso-based regression using a variety of values for $\alpha$ between 0 and 1 via a grid search where  *housing_labels* is the output and all other features are the input (similar to as seen in lecture two.)

In [0]:
categorical_vars = ['ocean_proximity']

discrete_vars = ['households', 'total_rooms', 'total_bedrooms','population']

continuous_vars = ['latitude', 'longitude','median_income','housing_median_age']

dep_var = ['median_house_value']

In [164]:
print(housing[['latitude']].values[:, 0])
tfmr = StandardScaler()
print(tfmr.fit_transform(housing[['latitude']].values)[:, 0])



[37.88 37.86 37.85 ... 39.43 39.43 39.37]
[1.05254828 1.04318455 1.03850269 ... 1.77823747 1.77823747 1.75014627]


In [165]:
print(housing[['median_income']].values[:, 0])
tfmr = StandardScaler()
print(tfmr.fit_transform(housing[['median_income']].values)[:, 0])

[8.3252 8.3014 7.2574 ... 1.7    1.8672 2.3886]
[ 2.34476576  2.33223796  1.7826994  ... -1.14259331 -1.05458292
 -0.78012947]


In [0]:
# Normalize all non-categorical data 
cat_ohe_step = ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
cat_steps = [cat_ohe_step]
cat_pipe = Pipeline(cat_steps)
cat_transformers = [('cat', cat_pipe, categorical_vars)]

# Numerical - impute, scale
num_si_step = ('si', SimpleImputer(strategy='median'))
num_scl_step = ('scl', MinMaxScaler())
num_steps = [num_si_step, num_scl_step]
num_pipe = Pipeline(num_steps)
num_transformers = [('num', num_pipe, discrete_vars + continuous_vars)]

ct = ColumnTransformer(transformers=cat_transformers + num_transformers)
ct.fit(housing[categorical_vars + discrete_vars + continuous_vars])
X = ct.transform(housing[categorical_vars + discrete_vars + continuous_vars])
   
    # It is also generally not neccessary to apply transformations to normalize or scale\n",
    # the dependent variable.\n",
y = housing[['median_house_value']].values

In [167]:
#convert all categorical to one hot vectors

print(housing[['ocean_proximity']].values)
tfmr = OneHotEncoder(sparse=False)
print(tfmr.fit_transform(housing[['ocean_proximity']].values))

[['NEAR BAY']
 ['NEAR BAY']
 ['NEAR BAY']
 ...
 ['INLAND']
 ['INLAND']
 ['INLAND']]
[[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 ...
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]


In [168]:
#split data into training and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(16512, 13) (4128, 13) (16512, 1) (4128, 1)


In [0]:
housing['lat_cat'] = np.ceil(housing['latitude']*10)

In [0]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

# note that we pass the dataframe as arguments to the .split() method 
# rather than the numpy matrix X. This is done for easy comprehension
# as the numpy matrix no longer has labelled columns. You can compare the
# shapes of X and df to ensure we are getting the right rows.
for train_index, test_index in split.split(housing, housing['lat_cat'].fillna(housing['lat_cat'].median())):
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

In [171]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
# Here below I have specified values of alpha which can be any value from 0 to 1
alphas = np.array([0.1, 0.3, 0.4, 0.5,1])

lasso = Lasso(normalize=True, max_iter=10000)

grid_search = GridSearchCV(estimator=lasso, param_grid=dict(alpha=alphas) , cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)

grid_search.fit(X_train, y_train)
print(grid_search.best_estimator_.alpha)
rmse = np.sqrt(-grid_search.best_score_)
print("rmse", rmse)

cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

Lassoreg = Lasso(max_iter=100000)
Lassoreg.fit(X_train, np.ravel(y_train))
train_score=Lassoreg.score(X_train, np.ravel(y_train))
test_score=Lassoreg.score(X_test,np.ravel(y_test))
coeff_used = np.sum(Lassoreg.coef_!=0)
print ("training score:", train_score)
print ("test score: ", test_score)

0.3
rmse 69047.99657499259
69048.3014005975 {'alpha': 0.1}
69047.99657499259 {'alpha': 0.3}
69048.16088358698 {'alpha': 0.4}
69048.53561966716 {'alpha': 0.5}
69053.55605994261 {'alpha': 1.0}
training score: 0.6436384767733125
test score:  0.6518257141259914


In [172]:
# Finding RSME of test set.
print(grid_search.best_estimator_.alpha)
rmse = np.sqrt(-grid_search.best_score_)
print (rmse)


0.3
69047.99657499259


**Q2. Why is it necessary to normalize all continuous variables before performing Lasso?**

<pre> To avoid higher absolute value variable to overweight, to find the global minimum and also decreases the runtime complexity, it necessary to normalize all continuous variables before performing Lasso regresssion</pre>

### Conclusions
**Q3. For what values of $\alpha$ does Lasso perform best? Does it perform as well on the housing data as the linear regressor from the lectures? Why do you think this is?**

<pre>For α=0.3 is best that performs Lasso. 
Our training score: 0.6436384767733125
test score: 0.6518257141259914
rmse value is 69047.99657499259.

We found out lasso regression is better, after the comparison between linear regression and lasso regression. </pre>




### Submit your notebook

Submit your solution on Canvas